# Введение в MapReduce модель на Python


In [32]:
from typing import NamedTuple # requires python 3.6+
from typing import Tuple
from typing import Iterator

In [33]:
def MAP(_, row: NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows: Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum / count)
  else:
    yield (age, 0)

Модель элемента данных

In [34]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [35]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [36]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [37]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [38]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [39]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [40]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [41]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [42]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [43]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [44]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [45]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row: NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age: str, rows: Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [46]:
from typing import Iterator
import numpy as np

mat = np.ones((5, 4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates: Tuple[int, int], value: int):
  i, j = coordinates
  yield (i, value * vec[j])

def REDUCE(i: int, products: Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i, j])

print(mat)
print(vec)
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]
[0.71706127 0.3712892  0.59395051 0.42852747]


[(0, 2.1108284504106596),
 (1, 2.1108284504106596),
 (2, 2.1108284504106596),
 (3, 2.1108284504106596),
 (4, 2.1108284504106596)]

## Inverted index

In [47]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId: str, body: str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word: str, docIds: Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('what', ['0', '1']),
 ('it', ['0', '1', '2']),
 ('is', ['0', '1', '2']),
 ('a', ['2']),
 ('banana', ['2'])]

## WordCount

In [48]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid, lineid), line)

def MAP(docId: str, line: str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word: str, counts: Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [49]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [50]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid, lineid), line)

  split_size = int(np.ceil(len(documents) / maps))

  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i + split_size])

def MAP(docId: str, line: str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word: str, counts: Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('a', 2), ('banana', 2), ('is', 18), ('what', 10)]),
 (1, [('it', 18)])]

## TeraSort

In [51]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for value in split:
        yield (value, None)

  split_size =  int(np.ceil(len(input_values) / maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])

def MAP(value:int, _):
  yield (value, None)

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.0011496019078764963),
   (None, 0.031218084388362155),
   (None, 0.050836165304943726),
   (None, 0.053620807159475814),
   (None, 0.06349275231993101),
   (None, 0.08867807211384926),
   (None, 0.11327564529212619),
   (None, 0.14917190758185284),
   (None, 0.18730430572942547),
   (None, 0.24549201366170648),
   (None, 0.28623373220722836),
   (None, 0.30329854557412994),
   (None, 0.3056506200933863),
   (None, 0.3162210044906526),
   (None, 0.3229974990395904),
   (None, 0.32998113137048524),
   (None, 0.33473049232728147),
   (None, 0.3364638960853792),
   (None, 0.3606674575089239),
   (None, 0.48951082517414124)]),
 (1,
  [(None, 0.5001093590246479),
   (None, 0.515635536894885),
   (None, 0.533740156646782),
   (None, 0.5444526246120444),
   (None, 0.610340468320169),
   (None, 0.7408524539841618),
   (None, 0.8937291789258196),
   (None, 0.9496949896775627),
   (None, 0.976627132858243),
   (None, 0.9880467054202186)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [52]:
from typing import Iterator
from typing import List
import numpy as np


def MAP(key: int, value: int):
    yield (1, value)

def REDUCE(key: int, row: List[int]):
    yield max(row)

def RECORDREADER():
    random_numbers = np.random.randint(10, size=100)
    indexed_numbers = [(index, num) for index, num in enumerate(random_numbers)]
    return indexed_numbers

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[9]

### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [53]:
import numpy as np
n = 10000

def MAP(key: int, value: int):
    yield (1, value)

def REDUCE(key: int, row: List[int]):
    yield np.mean(row)

def RECORDREADER():
    random_numbers = np.random.randint(10, size=100)
    indexed_numbers = [(index, num) for index, num in enumerate(random_numbers)]
    return indexed_numbers

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[4.38]

### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [54]:
import numpy as np

n = 10

def MAP(key: int, value: int):
    yield (1, value)

def RECORDREADER():
    random_numbers = np.random.randint(10, size=100)
    indexed_numbers = [(index, num) for index, num in enumerate(random_numbers)]
    return indexed_numbers

def flatten(iterable):
    return [item for sublist in iterable for item in sublist]

def groupbykey(iterable):
    t = {}

    for (k, v) in iterable:
        t[k] = t.get(k, []) + [v]

    return t.items()

def groupbykey_sorted(iterable):
    t = {}

    for (k2, v2) in iterable:
        t[k2] = sorted(t.get(k2, []) + [v2])

    return t.items()

mapped_data = flatten(map(lambda x: MAP(*x), RECORDREADER()))

print(groupbykey(mapped_data))
print(groupbykey_sorted(mapped_data))

dict_items([(1, [7, 4, 2, 7, 0, 5, 6, 0, 6, 3, 9, 6, 3, 4, 8, 7, 5, 8, 4, 5, 6, 0, 5, 4, 3, 4, 9, 4, 8, 8, 4, 7, 8, 0, 8, 3, 8, 3, 6, 2, 0, 7, 3, 8, 3, 3, 9, 1, 3, 5, 4, 5, 0, 5, 9, 3, 2, 8, 6, 7, 6, 8, 9, 9, 1, 8, 2, 3, 6, 2, 1, 6, 5, 0, 7, 3, 4, 0, 0, 1, 4, 7, 4, 7, 4, 6, 0, 7, 2, 0, 1, 1, 5, 7, 8, 3, 6, 9, 4, 8])])
dict_items([(1, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9])])


### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [55]:
from typing import List

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
        User(id=0, age=55, gender='male', social_contacts=20),
        User(id=1, age=25, gender='female', social_contacts=240),
        User(id=2, age=25, gender='female', social_contacts=500),
        User(id=3, age=33, gender='female', social_contacts=800),
        User(id=0, age=55, gender='male', social_contacts=20),
        User(id=1, age=25, gender='female', social_contacts=240),
        User(id=2, age=25, gender='female', social_contacts=500),
        User(id=3, age=33, gender='female', social_contacts=800)
    ]

def MAP(key: int, value: User):
    yield (value.id, value)

def REDUCE(key: int, values: List[User]):
    yield values[0]

def RECORDREADER():
    return [(index, user) for index, user in enumerate(input_collection)]

output = MapReduce(RECORDREADER, MAP, REDUCE)

for user in output:
    print(user)

User(id=0, age=55, social_contacts=20, gender='male')
User(id=1, age=25, social_contacts=240, gender='female')
User(id=2, age=25, social_contacts=500, gender='female')
User(id=3, age=33, social_contacts=800, gender='female')


#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [56]:
class User(NamedTuple):
    id: int
    age: str
    social_contacts: int
    gender: str

In [57]:
from typing import List, NamedTuple

def condition(user: User) -> bool:
    return user.gender == 'male'

def MAP(key: int, value: User):
    if condition(value):
        yield (value, value)

def REDUCE(key: int, values: List[User]):
    yield values[0]

input_collection = [
        User(id=0, age=55, gender='male', social_contacts=20),
        User(id=1, age=25, gender='female', social_contacts=240),
        User(id=2, age=25, gender='female', social_contacts=500),
        User(id=3, age=33, gender='female', social_contacts=800),
    ]


def RECORDREADER():
  return [(index, user) for index, user in enumerate(input_collection)]


output = MapReduce(RECORDREADER, MAP, REDUCE)

for element in output:
    print(element)


User(id=0, age=55, social_contacts=20, gender='male')


### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [58]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800),
    User(id=4, age=16, gender='male', social_contacts=420)
]

def MAP(key, value):
  yield (value.gender, value.gender)

def REDUCE(key, rows):
  yield (key, key)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)

for element in output:
    print(element)

('male', 'male')
('female', 'female')


### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [59]:
collection_1 = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800),
    User(id=4, age=16, gender='male', social_contacts=420),
]

collection_2 = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=3, age=33, gender='female', social_contacts=800),
]

def MAP(key, value):
  yield (value, value)

def REDUCE(key, rows):
  yield (key, key)

def RECORDREADER():
  return [(u.id, u) for u in collection_1 + collection_2]

output = MapReduce(RECORDREADER, MAP, REDUCE)

for element in output:
    print(element)

(User(id=0, age=55, social_contacts=20, gender='male'), User(id=0, age=55, social_contacts=20, gender='male'))
(User(id=1, age=25, social_contacts=240, gender='female'), User(id=1, age=25, social_contacts=240, gender='female'))
(User(id=2, age=25, social_contacts=500, gender='female'), User(id=2, age=25, social_contacts=500, gender='female'))
(User(id=3, age=33, social_contacts=800, gender='female'), User(id=3, age=33, social_contacts=800, gender='female'))
(User(id=4, age=16, social_contacts=420, gender='male'), User(id=4, age=16, social_contacts=420, gender='male'))


### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [60]:
first_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800),
    User(id=4, age=16, gender='male', social_contacts=420),
]
second_collection = [
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=4, age=16, gender='male', social_contacts=420),
    User(id=6, age=23, gender='male', social_contacts=228),
]

def MAP(key, value):
  yield (value, value)

def REDUCE(key, rows):
  if len(rows) > 1:
    yield (key, key)

def RECORDREADER():
  return [(u.id, u) for u in first_collection + second_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)

for element in output:
    print(element)

(User(id=2, age=25, social_contacts=500, gender='female'), User(id=2, age=25, social_contacts=500, gender='female'))
(User(id=4, age=16, social_contacts=420, gender='male'), User(id=4, age=16, social_contacts=420, gender='male'))


### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [61]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

In [62]:
first_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800),
    User(id=4, age=16, gender='male', social_contacts=420),
]

second_collection = [
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=4, age=16, gender='male', social_contacts=420),
    User(id=6, age=23, gender='male', social_contacts=228),
]

def contains(list, item):
    for el in list:
      if el is item:
        return True
    return False

def MAP(key, value):
  if contains(first_collection, value):
    yield (value, 0)
  elif contains(second_collection, value):
    yield (value, 1)

def REDUCE(key, rows):
  if (rows == [0]):
    yield (key, key)

def RECORDREADER():
  return [(u.id, u) for u in first_collection + second_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(sorted(output))
output

[(User(id=0, age=55, social_contacts=20, gender='male'),
  User(id=0, age=55, social_contacts=20, gender='male')),
 (User(id=1, age=25, social_contacts=240, gender='female'),
  User(id=1, age=25, social_contacts=240, gender='female')),
 (User(id=3, age=33, social_contacts=800, gender='female'),
  User(id=3, age=33, social_contacts=800, gender='female'))]

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [63]:
class Department(NamedTuple):
  department_name: str
  manager_name: str

class PartialDepartment(NamedTuple):
  manager_name: str

class Employee(NamedTuple):
  id: int
  name: str
  department_name: str

class PartialEmployee(NamedTuple):
  id: int
  name: str

class EmployeeDepartment(NamedTuple):
  id: int
  name: str
  department_name: str
  manager_name: str

department_collection = [
    Department(department_name="IT", manager_name="Bob"),
    Department(department_name="Sales", manager_name="James"),
    Department(department_name="HR", manager_name="Anna"),
    Department(department_name="Finance", manager_name="William"),
    Department(department_name="Marketing", manager_name="Ysabell")
]

employee_collection = [
   Employee(id=1, name="Thomas", department_name="IT"),
   Employee(id=2, name="Adam", department_name="IT"),
   Employee(id=3, name="Kevin", department_name="Sales"),
   Employee(id=4, name="Brian", department_name="HR"),
   Employee(id=5, name="George", department_name="Finance"),
   Employee(id=6, name="Scott", department_name="Marketing"),
]

def MAP(key, value):
  if value in employee_collection:
    yield (value.department_name, (0, PartialEmployee(id=value.id, name=value.name)))
  elif value in department_collection:
    yield (value.department_name, (1, PartialDepartment(manager_name=value.manager_name)))

def REDUCE(key, rows):
  for i in range(len(rows)):
    for j in range(len(rows)):
      if (rows[i][0] == 1 and rows[j][0] == 0):
        yield (key, EmployeeDepartment(id=rows[j][1].id, name=rows[j][1].name, department_name=key, manager_name=rows[i][1].manager_name))

def RECORDREADER():
  return [(index, u) for index, u in enumerate(department_collection + employee_collection)]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(sorted(output))
output

[('Finance',
  EmployeeDepartment(id=5, name='George', department_name='Finance', manager_name='William')),
 ('HR',
  EmployeeDepartment(id=4, name='Brian', department_name='HR', manager_name='Anna')),
 ('IT',
  EmployeeDepartment(id=1, name='Thomas', department_name='IT', manager_name='Bob')),
 ('IT',
  EmployeeDepartment(id=2, name='Adam', department_name='IT', manager_name='Bob')),
 ('Marketing',
  EmployeeDepartment(id=6, name='Scott', department_name='Marketing', manager_name='Ysabell')),
 ('Sales',
  EmployeeDepartment(id=3, name='Kevin', department_name='Sales', manager_name='James'))]

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [64]:
class Home(NamedTuple):
  id: int
  home_type: str
  price: int

rental_collection = [
    Home(id=1, home_type="Private room", price=149),
    Home(id=2, home_type="Entier home/apt", price=225),
    Home(id=3, home_type="Private room", price=150),
    Home(id=4, home_type="Entier home/apt", price=89),
    Home(id=5, home_type="Entier home/apt", price=80),
]

# SELECT home_type, AVG(price) FROM rental_collection GROUP BY home_type

def MAP(key, value):
  yield (value.home_type, value.price)

def REDUCE(key, rows):
  yield (key, sum(rows) / len(rows))

def RECORDREADER():
  return [(index, u) for index, u in enumerate(rental_collection)]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(sorted(output))
output

[('Entier home/apt', 131.33333333333334), ('Private room', 149.5)]

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи

http://infolab.stanford.edu/~ullman/mmds/book.pdf page 50


In [65]:
ROWS = 5
COLS = 4
STRIPES = 2

COLS_IN_STRIPE = COLS // STRIPES


M = [np.ones((ROWS, COLS_IN_STRIPE)) for i in range(STRIPES)]

v = [np.random.rand(COLS_IN_STRIPE) for i in range(STRIPES)]


def MAP(coordinates: Tuple[int, int], value: int):
  i, j = coordinates
  yield (i, value * vec[j])

def REDUCE(i: int, products: Iterator[NamedTuple]):
  print(i, products)
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  global vec
  for stripe_index, stripe in enumerate(M):
    vec = v[stripe_index]
    for i in range(stripe.shape[0]):
      for j in range(stripe.shape[1]):
        yield ((i, j), stripe[i, j])

print(M)
print(v)
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]]), array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]])]
[array([0.00872527, 0.51527751]), array([0.26183578, 0.38090398])]
0 [0.008725274872762645, 0.5152775090850913, 0.2618357835707126, 0.38090398247121704]
1 [0.008725274872762645, 0.5152775090850913, 0.2618357835707126, 0.38090398247121704]
2 [0.008725274872762645, 0.5152775090850913, 0.2618357835707126, 0.38090398247121704]
3 [0.008725274872762645, 0.5152775090850913, 0.2618357835707126, 0.38090398247121704]
4 [0.008725274872762645, 0.5152775090850913, 0.2618357835707126, 0.38090398247121704]


[(0, 1.1667425499997837),
 (1, 1.1667425499997837),
 (2, 1.1667425499997837),
 (3, 1.1667425499997837),
 (4, 1.1667425499997837)]

## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [66]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [67]:
I = 2
J = 3
K = 4 * 10
small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

def RECORDREADER():
  for j in range(J):
    for k in range(K):
      yield ((j, k), big_mat[j, k])

def MAP(k1, v1):
  (j, k) = k1
  w = v1
  for i in range(I):
    yield ((i, k), small_mat[i, j] * w)

def REDUCE(key, values):
  (i, k) = key
  yield ((i, k), sum(values))

Проверьте своё решение

In [68]:
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i, k), vw) in reduce_output) + 1
  K = max(k for ((i, k), vw) in reduce_output) + 1
  mat = np.empty(shape=(I, K))
  for ((i, k), vw) in reduce_output:
    mat[i, k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution))

True

In [69]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i, k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

http://infolab.stanford.edu/~ullman/mmds/book.pdf page 55

In [70]:
I = 2
J = 3
K = 4 * 10
small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

def RECORDREADER():
  for i in range(I):
    for j in range(J):
      yield ((0, i, j), small_mat[i, j])

  for j in range(J):
    for k in range(K):
      yield ((1, j, k), big_mat[j, k])

def MAP(k1, v1):
  (matrix_index, row_index, col_index) = k1
  if matrix_index == 0:
    yield (col_index, (matrix_index, row_index, v1))
  else:
    yield (row_index, (matrix_index, col_index, v1))


def REDUCE(key, values):
  first_matrix = [el for el in values if el[0] == 0]
  second_matrix = [el for el in values if el[0] == 1]

  for first_el in first_matrix:
    for second_el in second_matrix:
      (_, first_index, first_value) = first_el
      (_, second_index, second_value) = second_el
      yield ((first_index, second_index), first_value * second_value)


def MAP_IDENTITY(*args):
  yield args

def REDUCE_AGREGATE(key, values):
  yield (key, sum(values))

def GET_RECORDREADER(iterable):
  def func():
    for el in iterable:
      yield el
  return func


reference_solution = np.matmul(small_mat, big_mat)
partial_solution = MapReduce(RECORDREADER, MAP, REDUCE)
solution = MapReduce(GET_RECORDREADER(partial_solution), MAP_IDENTITY, REDUCE_AGREGATE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i, k), vw) in reduce_output) + 1
  K = max(k for ((i, k), vw) in reduce_output) + 1
  mat = np.empty(shape=(I, K))
  for ((i, k), vw) in reduce_output:
    mat[i, k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [71]:
I = 2
J = 3
K = 4 * 10
small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

maps = 2
reducers = 2

def INPUTFORMAT():
  def RECORDREADER(index, matrix):
    rows, cols = matrix.shape
    for i in range(rows):
      for j in range(cols):
        yield ((index, i, j), matrix[i, j])

  yield RECORDREADER(0, small_mat)
  yield RECORDREADER(1, big_mat)

def MAP(k1, v1):
  (matrix_index, row_index, col_index) = k1

  if matrix_index == 0:
    yield (col_index, (matrix_index, row_index, v1))
  else:
    yield (row_index, (matrix_index, col_index, v1))

def REDUCE(key, values):
  first_matrix = [el for el in values if el[0] == 0]
  second_matrix = [el for el in values if el[0] == 1]

  for first_el in first_matrix:
    for second_el in second_matrix:
      (_, first_index, first_value) = first_el
      (_, second_index, second_value) = second_el
      yield ((first_index, second_index), first_value * second_value)

def MAP_IDENTITY(*args):
  yield args

def REDUCE_AGREGATE(key, values):
  yield (key, sum(values))

def GET_INPUTFORMAT(iterable):
  def func():
    for el in iterable:
      yield el[1]
  return func

reference_solution = np.matmul(small_mat, big_mat)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]

solution = MapReduceDistributed(GET_INPUTFORMAT(partitioned_output), MAP_IDENTITY, REDUCE_AGREGATE, COMBINER=None)
solution = [(partition_id, list(partition)) for (partition_id, partition) in solution]

solution_unpacked = []
for value in solution:
  for v in value[1]:
    solution_unpacked.append(v)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i, k), vw) in reduce_output) + 1
  K = max(k for ((i, k), vw) in reduce_output) + 1
  mat = np.empty(shape=(I, K))
  for ((i, k), vw) in reduce_output:
    mat[i, k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution_unpacked)) # should return true

126 key-value pairs were sent over a network.
240 key-value pairs were sent over a network.


True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [72]:
I = 2
J = 3
K = 4
small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

maps = 2
reducers = 2

def INPUTFORMAT():
  def RECORDREADER(index, matrix, shift):
    rows, cols = matrix.shape
    for i in range(rows):
      for j in range(cols):
        yield ((index, i + shift, j), matrix[i, j])

  split_size = int(np.ceil(len(small_mat) / maps))

  for i in range(0, len(small_mat), split_size):
    yield RECORDREADER(0, small_mat[i:i + split_size], i)

  split_size = int(np.ceil(len(big_mat) / maps))

  for i in range(0, len(big_mat), split_size):
    yield RECORDREADER(1, big_mat[i:i + split_size], i)

def MAP(k1, v1):
  (matrix_index, row_index, col_index) = k1

  if matrix_index == 0:
    yield (col_index, (matrix_index, row_index, v1))
  else:
    yield (row_index, (matrix_index, col_index, v1))

def REDUCE(key, values):
  first_matrix = [el for el in values if el[0] == 0]
  second_matrix = [el for el in values if el[0] == 1]

  for first_el in first_matrix:
    for second_el in second_matrix:
      (_, first_index, first_value) = first_el
      (_, second_index, second_value) = second_el
      yield ((first_index, second_index), first_value * second_value)

def MAP_IDENTITY(*args):
  yield args

def REDUCE_AGREGATE(key, values):
  yield (key, sum(values))

def GET_INPUTFORMAT(iterable):
  def func():
    for el in iterable:
      yield el[1]
  return func

reference_solution = np.matmul(small_mat, big_mat)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]

solution = MapReduceDistributed(GET_INPUTFORMAT(partitioned_output), MAP_IDENTITY, REDUCE_AGREGATE, COMBINER=None)
solution = [(partition_id, list(partition)) for (partition_id, partition) in solution]

solution_unpacked = []
for value in solution:
  for v in value[1]:
    solution_unpacked.append(v)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i, k), vw) in reduce_output) + 1
  K = max(k for ((i, k), vw) in reduce_output) + 1
  mat = np.empty(shape=(I, K))
  for ((i, k), vw) in reduce_output:
    mat[i, k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution_unpacked)) # should return true

18 key-value pairs were sent over a network.
24 key-value pairs were sent over a network.


True